# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test` __Do NOT commit the pickle file__

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [16]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)

    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df

    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    while(True):
        yield df.sample(100).reset_index(drop=True)

gen = partial_df()

In [17]:
X_test, y_test = test_df()

In [18]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,NaN,NaN,NaN,NaN,automatic,3gnal4ek9fs503144,fl,4,16396.0,blue,black,gm remarketing,19250,19300,Mon Feb 23 2015 05:00:00 GMT-0800 (PST)
1,2005,Nissan,Quest,3.5 SE,Minivan,automatic,5n1bv28u05n111490,md,2.1,999999.0,white,beige,355 toyota,1725,1100,Tue Jan 13 2015 09:30:00 GMT-0800 (PST)
2,2014,Chevrolet,Malibu,LT,Sedan,automatic,1g11e5sl4ef133318,tn,4.3,41926.0,black,gray,enterprise veh exchange/rental,14000,14600,Wed Feb 11 2015 02:30:00 GMT-0800 (PST)
3,2012,Toyota,Camry,XLE,Sedan,automatic,4t4bf1fk3cr180871,wi,2.7,27981.0,red,beige,toyota financial services,16300,15500,Wed Mar 04 2015 02:00:00 GMT-0800 (PST)
4,2011,Dodge,Charger,Base,Sedan,automatic,2b3cl3cg7bh520727,md,3.8,28437.0,silver,black,exeter finance corp,14300,14000,Tue Feb 24 2015 01:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2012,Infiniti,M,M37x,Sedan,NaN,jn1by1ar1cm392135,nj,2.5,51417.0,blue,black,nissan infiniti lt,23800,24750,Wed Jan 07 2015 09:30:00 GMT-0800 (PST)
96,2014,Ford,Explorer,XLT,suv,NaN,1fm5k8d8xegc50581,co,4.2,33001.0,—,brown,avis budget group,26500,27200,Wed Jun 17 2015 04:15:00 GMT-0700 (PDT)
97,2013,Dodge,Avenger,SXT,sedan,automatic,1c3cdzcb5dn625459,fl,3.3,56738.0,blue,black,the hertz corporation,9375,9200,Tue Jun 16 2015 02:15:00 GMT-0700 (PDT)
98,2003,Ford,Expedition,XLT Premium,SUV,NaN,1fmpu16l53lc39424,nj,2.8,128222.0,green,gray,paramus ford inc,2450,3600,Wed Jan 07 2015 09:30:00 GMT-0800 (PST)


In [21]:
from sklearn.linear_model import SGDRegressor
from sklearn.impute import SimpleImputer
regressor = SGDRegressor(max_iter=1000, tol=1e-3)


batch_gen = partial_df()
for _ in range(50):
    X_batch = next(batch_gen)[['year', 'odometer']]
    y_batch = next(batch_gen)['sellingprice']

    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    X_batch_imputed = imputer.fit_transform(X_batch)

    regressor.partial_fit(X_batch_imputed, y_batch)

X_test, y_test = test_df()


imputer = SimpleImputer(strategy='median')
X_test_imputed = imputer.fit_transform(X_test[['year', 'odometer']])

test_predictions = regressor.predict(X_test_imputed)

# Calculate performance metrics (e.g., RMSE)
rmse = np.sqrt(np.mean((test_predictions - y_test) ** 2))
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 1.042741814218349e+18


In [22]:
test_predictions

array([1.18390468e+18, 4.32193100e+17, 2.59872654e+17, ...,
       2.78544842e+17, 7.51431785e+16, 2.23745458e+17])